## データとライブラリの読み込み

In [23]:
import pandas as pd
import pandas_profiling
import numpy as np

#google drive読み込み
# from google.colab import drive, files
# drive.mount('/content/drive')
# dataPath = "/content/drive/My Drive/01 Lab/novel_bookmark/datasets/"
# test = pd.read_csv(dataPath + 'test.csv')
# train = pd.read_csv(dataPath +'train.csv')
# sub = pd.read_csv(dataPath +'sample_submission.csv')

#local 読み込み
dataPath = "datasets/"
test = pd.read_csv(dataPath + 'test.csv')
train = pd.read_csv(dataPath +'train.csv')
sub = pd.read_csv(dataPath +'sample_submission.csv')

## データの可視化、確認

In [24]:
#データの可視化
# !pip install pandas-profiling
# !pip install --upgrade pandas_profiling
# train.profile_report()

In [25]:
df = train
print(len(df[train['fav_novel_cnt_bin']==4]))


sub = df[train['fav_novel_cnt_bin']==4]
sub.to_csv('pre_file.csv', index=False)
files.download('pre_file.csv')

814


NameError: name 'files' is not defined

## 特徴量エンジニアリング

In [ ]:
test = pd.read_csv(dataPath + 'test.csv')
train = pd.read_csv(dataPath +'train.csv')
sub = pd.read_csv(dataPath +'sample_submission.csv')

data = pd.concat([train, test], sort=False)#データの統合

#必要のないカラムを削除
#ncode:小説コード, writer, userid, pc_or_k:小説の投稿機種 
delete_columns = ['ncode','writer', 'userid', 'pc_or_k']
data.drop(delete_columns, axis=1, inplace=True)

#novel_type (1, 2)を(0, 1)に変更
data['novel_type'] = data['novel_type'].map({1: 0, 2: 1})

#'general_firstup'(初回投稿日)を今日までの日数に変換
import datetime
def calc_diff_days(x):#return days(int)
  y_m_d = x.split(" ")[0]
  y, m, d = map(int, y_m_d.split("-"))
  ymd = datetime.datetime(y, m, d)
  diff = datetime.datetime.today() - ymd
  return diff.days
data['general_firstup'] = data['general_firstup'].map(calc_diff_days)

#一時的にテキストデータを削除
text_columns = ['title','story', 'keyword']
data.drop(text_columns, axis=1, inplace=True)

#dataをtest, trainに分割
train = data[:len(train)]
test = data[len(train):]

train

,general_firstup,biggenre,genre,novel_type,end,isstop,isr15,isbl,isgl,iszankoku,istensei,istenni,fav_novel_cnt_bin
0,5306,4,402,0,0,0,0,0,0,0,0,0,1.0
1,5285,4,401,0,0,0,0,0,0,0,0,0,2.0
2,5163,4,403,0,0,0,0,0,0,0,0,0,3.0
3,5148,4,402,0,0,0,0,0,0,0,0,0,1.0
4,5137,4,401,0,0,0,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,59,99,9903,1,0,0,0,0,0,0,0,0,2.0
39996,59,1,102,1,0,0,0,0,0,0,0,0,0.0
39997,59,3,307,1,0,0,0,0,0,0,0,0,0.0
39998,59,2,201,1,0,0,0,0,0,0,0,0,2.0


### 訓練データと答えに分割

In [ ]:
x_train = train.drop('fav_novel_cnt_bin', axis=1)#訓練データ
y_train = train['fav_novel_cnt_bin']#訓練データの答え
x_test = test#テストデータ

x_train = train[:35000].drop('fav_novel_cnt_bin', axis=1)
y_train = train[:35000]['fav_novel_cnt_bin']
x_test = train[35000:].drop('fav_novel_cnt_bin', axis=1)
y_test = train[35000:]['fav_novel_cnt_bin']

print(len(x_train))
print(len(x_test))

35000
5000


## 勾配ブースティング


In [ ]:
#勾配ブースティング
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=0, stratify=y_train)
categorical_features = ['biggenre', 'genre']

import lightgbm as lgb
lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(x_valid, y_valid, categorical_feature=categorical_features)

params = {
    'objective': 'binary'
}

model = lgb.train(params,
                  lgb_train,
                  valid_sets=[lgb_train, lgb_eval],
                  verbose_eval=10,
                  num_boost_round=1000,
                  early_stopping_rounds=10
                  )
pred = model.predict(x_test, num_iteration=model.best_iteration)

C:\Users\ngnls\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ngnls\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\ngnls\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\ngnls\AppData\Loca

[LightGBM] [Info] Number of positive: 13196, number of negative: 11304
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001952 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 297
[LightGBM] [Info] Number of data points in the train set: 24500, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.538612 -> initscore=0.154757
[LightGBM] [Info] Start training from score 0.154757
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.56765	valid_1's binary_logloss: 0.575766
[20]	training's binary_logloss: 0.542886	valid_1's binary_logloss: 0.555961
[30]	training's binary_logloss: 0.534779	valid_1's binary_logloss: 0.551555
[40]	training's binary_logloss: 0.53	valid_1's binary_logloss: 0.550059
[50]	training's binary_logloss: 0.526245	valid_1's binary_logloss: 0.549549
[60]	t

5001
35000                                                              0.0
35001                                                              2.0
35002                                                              0.0
35003                                                              0.0
35004                                                              1.0
                                           ...                        
39996                                                              0.0
39997                                                              0.0
39998                                                              2.0
39999                                                              0.0
fav_novel_cnt_bin    [3.0, 4.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, ...
Name: fav_novel_cnt_bin, Length: 5001, dtype: object


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets